# LightGBM の実装

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import datetime
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import lightgbm as lgb
from catboost import CatBoostClassifier

In [2]:
!mkdir oof
!mkdir models

mkdir: oof: File exists
mkdir: models: File exists


In [3]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    DATA_PATH = Path('.')
    OOF_DATA_PATH = Path('./oof')
    MODEL_DATA_PATH = Path('./models')
    METHOD_LIST = ['catboost', 'lightgbm']
    seed = random.randint(0, 100)
    n_folds = 5
    target_col = 'price'
    USE_PLAYER_FEATURES = False
    num_boost_round = 50500
    early_stopping_round = 50
    verbose = 2000
    boosting_type = 'gbdt'
    lgb_params = {
        'objective':'classifier',
        'boosting': boosting_type,
        'n_jobs': -1,
        'seed': seed,
        'force_col_wise':'true',
        'num_boost_round': num_boost_round
        }
    cat_params = {
        'iterations': num_boost_round,
        'random_seed': seed,
        }

In [4]:
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)

In [5]:
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame):
    # Apply the encoding map to the relevant columns
    categorical_cols = x_train.select_dtypes(include=['object']).columns.tolist()

    # Convert categorical columns to 'category' dtype
    for col in categorical_cols:
        x_train[col] = x_train[col].astype('category')
        x_valid[col] = x_valid[col].astype('category')

    # Create LightGBM datasets
    train_data = lgb.Dataset(x_train, label=y_train, categorical_feature=categorical_cols)
    valid_data = lgb.Dataset(x_valid, label=y_valid, categorical_feature=categorical_cols)

    # Train model
    model = lgb.train(
        params=CFG.lgb_params,
        train_set=train_data,
        num_boost_round=20000,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(CFG.early_stopping_round, verbose=True), lgb.log_evaluation(CFG.verbose)]
        )

    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, categorical_features: list):
    # Define model
    model = CatBoostClassifier(**CFG.cat_params, cat_features=categorical_features)

    # Fit model
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], early_stopping_rounds=CFG.early_stopping_round, verbose=CFG.verbose)

    # Predict validation
    valid_pred = model.predict(x_valid)

    return model, valid_pred

def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df, train_df[CFG.target_col])):
        print('-'*50)
        print(f'{method} training fold {fold + 1}')
        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, categorical_features)

        # Save best model
        pickle.dump(model, open(CFG.MODEL_DATA_PATH / f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'wb'))
        # Save encoders only for lightgbm and xgboost
        # if method in ['lightgbm', 'xgboost']:
        #     pickle.dump(encoders[fold], open(CFG.MODEL_DATA_PATH / f'{method}_encoders_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'wb'))

        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = np.sqrt(mean_squared_error(train_df[CFG.target_col], oof_predictions))
    print(f'{method} our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    # oof_df = pd.DataFrame({'id': train_df['id'], CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    # oof_df.to_csv(CFG.MODEL_DATA_PATH / f'oof_{method}_seed{CFG.seed}_ver{CFG.boosting_type}.csv', index = False)

In [ ]:
train_df = pd.read_csv(CFG.DATA_PATH / 'train.csv')
test_df = pd.read_csv(CFG.DATA_PATH / 'test.csv')
test_df[CFG.target_col] = -1
submission_df = pd.read_csv(CFG.DATA_PATH / 'submit_sample.csv')
all_df = pd.concat([train_df, test_df])

In [ ]:
import model_tuner
# all_df = model_tuner.select_features(all_df)

# 最後に、訓練データとテストデータに再度分割します
train_df = all_df[all_df['price'] != -1]
test_df = all_df[all_df['price'] == -1]

In [ ]:
train_df.info(verbose=True)

In [ ]:
# 'Id'や'Target'といった特定のカラムを除外した全てのカラムを特徴量とする場合
categorical_features = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
features = train_df.columns.drop(['id', 'price'])

In [ ]:
for method in CFG.METHOD_LIST:
    gradient_boosting_model_cv_training(method, train_df, features, categorical_features)

In [ ]:
def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        categorical_cols = x_test.select_dtypes(include=['object', 'category']).columns
        for col in categorical_cols:
            x_test[col] = x_test[col].astype('category')
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list):
    x_test = test_df[features]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test)
    if method == 'catboost':
        test_pred = catboost_inference(x_test)
    return test_pred


for method in CFG.METHOD_LIST:
    test_df[f'{method}_pred'] = gradient_boosting_model_inference(method, test_df, features)


# アンサンブルの結果を保存
test_df['final_pred'] = 0.5 * test_df['lightgbm_pred'] + 0.5 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/EMS_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# LGBのみの結果を保存
test_df['final_pred'] = 1.0 * test_df['lightgbm_pred'] + 0.0 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/OnlyLGB_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# Catのみの結果を保存
test_df['final_pred'] = 0.0 * test_df['lightgbm_pred'] + 1.0 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/OnlyCat_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)